# Introduction to Recurrent Neural Networks (RNNs)

## Learning stock embeddings for portfolio optimization using bidirectional RNNs

In [1]:
#Import dependencies
import numpy as np
import pandas as pd
import matplotlib as plt
import tensorflow as tf

## Recurrent Neural Networks (RNNs)

## Bidirectional Gated Recurrent Units (Bi-GRUs)

## Using Bi-GRUs for price movement classification

For the purposes of this assignment, we will focus on training a classifier for 15 stocks from the S&P 500. The goal of our classifier is as follows:
We are interested in training a bidirectional RNN model that learns a relationship between news taglines related to the 15 stocks $\{l_1, \ldots, l_{15}\}$ that we have selected and the prices of those stocks. Define $p_i^{(t)}$ to be the price of stock $l_i$ on day $t$. Then, we can formally define our objective as follows:

Let $y_i^{(t)} = \begin{cases} 1 & p_i^{(t)} \geq p_i^{(t - 1)} \\ 0 & p_i^{(t)} < p_i^{(t - 1)} \end{cases}$. Suppose our dataset $D = \{N^{(t)}\}_{t_{in} \leq t \leq t_f}$, where $N^{(t)}$ is a collection of all the articles from day $t$ and $t_{in}$ and $t_f$ represent the dates of the earliest and latest articles in our dataset resepctively. Then, we want to learn a mapping $\hat y_i^{(t)} = f(N^{(t - \mu)} \cup \ldots \cup N^{(t)})$ such that $\hat y_i^{(t)}$ accurately predicts $y_i^{(t)}$. More specifically, as is often the case with classification problems, we want to minimize the loss function given by the mean cross-entropy loss for all $15$ stocks:
$$\mathcal{L} = \frac{1}{15} \sum_{i = 1}^{15} \mathcal{L}_i = \frac{1}{15} \sum_{i = 1}^{15} \left( \frac{-1}{t_f - t_{in}} \sum_{t = t_{in}}^{t_f} \big(y_i^{(t)} \log \hat y_i^{(t)} + (1 - y_i^{(t)}) \log (1 - \hat y_i^{(t)}) \right)$$
Here, we choose to use $\mu = 4$, so we aim to classify the price movement of stock $l_i$ on day $t$, given by $p_i^{(t)}$, using news information from days $[t-4, t]$, i.e., articles $\{N^{(t - 4)}, N^{(t - 3)}, N^{(t - 2)}, N^{(t - 1)}, N^{(t)}\}$. Notice that we are including information from day $t$, so we are not *predicting* the price movement but rather identifying a relationship between the stock price movement and the information contained in the news taglines from day $t$ and the previous 4 days.

## Generating word embeddings

The code below loads word embeddings that we have pre-generated for 15 stocks from the S&P 500. We used news tagline data from Reuters (data sourced from https://github.com/vedic-partap/Event-Driven-Stock-Prediction-using-Deep-Learning/blob/master/input/news_reuters.csv) to create word embeddings for all of the articles in our dataset using a pretrained Spacy encoder and a Word2Vec model that we trained on our data (don't worry if you don't know what this means yet). Our dataset contains news articles from 2011 to 2017 so we should have enough data to build a fairly accurate classifier. You will explore algorithms for generating word embeddings in more detail later in the course but for this assignment, we have done the work for you so that you can focus on building RNN models for your stock movement classifier.

For the purposes of our classifier, we are focusing on the 15 stocks from the Reuters dataset for which we have the most data, i.e., news articles.

<br>

The main idea is to convert all of the qualitative textual information that we have in each article tagline into a quantitative feature that we can use when training our classifier. Let $s_i \in \mathbb{R}^{64}$ represent the stock embedding that we are trying to learn for stock $l_i$. We then define the following quantities:

Let $n_i^{(t)}$ be a news article from day $t$, for some $1 \leq i \leq |N^{(t)}|$. We associate 2 embedding vectors $K_i^{(t)} \in \mathbb{R}^{64}$ and $V_i^{(t)} \in \mathbb{R}^{300}$ with the article $n_i^{(t)}$, which we have computed for you below. We define $score(n_i^{(t)}, s_j) = K_i^{(t)} \cdot s_j$ and the softmax variable $$\alpha_i^{(t)} = \frac{\exp(score(n_i^{(t)}, s_j))}{\sum_{n_k^{(t)} \in N^{(t)}}exp(score(n_k^{(t)}, s_j))}$$

Finally, we define the market status of stock $m_j$ on day $t$, given by $m_j^{(t)} = \sum_{n_i^{(t)} \in N^{(t)}} \alpha_i^{(t)} V_i^{(t)}$. This is the input to the classifier that you will build and train on the dataset to learn the stock embeddings $\{s_j\}_{1 \leq j \leq 15}$.

In [2]:
data = pd.read_csv("embeddings.csv")
data

,index,Ticker,Name,Date,Headline,Tagline,Rating,K0,K1,K2,...,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299
0,1074,AAPL,1-800 FLOWERSCOM Inc,20140414,Apple antitrust compliance off to a promising ...,"NEW YORK Apple Inc has made a ""promising start...",topStory,0.728133,0.074376,-0.844244,...,-0.184006,0.032116,0.032128,-0.045440,0.027079,-0.100620,0.032597,-0.092093,0.048542,0.109286
1,1075,AAPL,1-800 FLOWERSCOM Inc,20140414,Apple antitrust compliance off to a promising ...,"NEW YORK April 14 Apple Inc has made a ""promi...",normal,0.757790,0.111567,-0.802569,...,-0.168789,0.039603,0.021292,-0.036883,0.029685,-0.110353,0.025347,-0.084554,0.045670,0.105747
2,1076,AAPL,1-800 FLOWERSCOM Inc,20140414,COLUMN-How to avoid the trouble coming to the ...,(The opinions expressed here are those of the ...,normal,-0.624152,-0.346050,-1.487509,...,-0.141506,-0.027039,-0.080825,-0.133556,0.018669,-0.056828,-0.052640,-0.169819,-0.033054,0.053817
3,1077,AAPL,1-800 FLOWERSCOM Inc,20140414,How to avoid the trouble coming to the tech se...,CHICAGO A resounding shot across the bow has b...,normal,0.387120,-0.099557,-0.590867,...,-0.233473,0.095700,0.113241,-0.027537,-0.119434,-0.074786,-0.072007,-0.049933,0.014863,0.063664
4,1078,AAPL,1-800 FLOWERSCOM Inc,20140415,Apple cannot escape U.S. states' e-book antitr...,NEW YORK Apple Inc on Tuesday lost an attempt ...,normal,0.824634,-1.637257,-0.352775,...,-0.232241,0.027836,-0.025965,0.036613,-0.087056,-0.103006,0.076729,-0.153311,0.038894,0.138866
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50787,184859,TAPR,Barclays Inverse US Treasury Composite ETN,20170209,BRIEF-Ultra Petroleum says Barclays agreed to ...,* Ultra Petroleum- on Feb 8 in connection wit...,normal,1.139437,0.682006,0.029171,...,-0.244216,0.053853,-0.008725,-0.048169,-0.032766,-0.062842,-0.059161,-0.104091,0.010547,0.129130
50788,184860,TAPR,Barclays Inverse US Treasury Composite ETN,20170209,MOVES-Barclays Nasdaq RenCap AXA BC Partners,Feb 9 The following financial services industr...,topStory,1.017802,-0.165982,-0.467275,...,-0.234947,0.049924,0.064670,0.022008,0.025572,-0.144732,-0.046366,-0.030195,-0.027131,0.093039
50789,184861,TAPR,Barclays Inverse US Treasury Composite ETN,20170217,Barclays Citi gave South Africa watchdog info...,JOHANNESBURG Feb 17 Barclays Plc and Citigrou...,normal,1.044449,-0.042930,0.201579,...,-0.234416,-0.001098,-0.035648,-0.053637,0.030076,-0.037331,0.048593,-0.019262,-0.030251,0.178724
50790,184862,TAPR,Barclays Inverse US Treasury Composite ETN,20170217,Barclays Citi helped South Africa with forex ...,JOHANNESBURG Barclays Plc and Citigroup appr...,topStory,1.288937,-0.372697,0.197727,...,-0.247672,0.049712,0.028656,-0.078167,0.047243,0.061589,0.016127,-0.073754,-0.011532,0.154577


Here, each row represents a different news article and is associated with one of the top 15 stocks that we are interested in for our classifier: <br>
`['AAPL', 'AMZN', 'BA', 'BCS', 'BP', 'C', 'DB', 'GM', 'GS', 'HSEA', 'HSEB', 'JPM', 'MSFT', 'MS', 'TAPR']`.

Additionally, the columns `[K0, ..., K63]` represent the components of the $K_i^{(t)}$ embedding vector and the columns `[V0, ..., V299]` represent the components of the $V_i^{(t)}$ embedding vector for each article $n_i^{(t)}$.

## Testing our Word Embeddings

## Building a Bi-GRU price movement classifier

### 1) a) Data processing

In [3]:
## do it for one stock, AAPL
aapl = data[data['Ticker'] == 'AAPL']
len(aapl)

6674

In [4]:
## set kappa to be max number of articles for a given day
kappa = np.max(aapl.groupby('Date').count()['index'])
kappa

12

In [5]:
## remove dates that have < 4 articles
drop_dates = set(aapl['Date'].unique()[(aapl.groupby('Date').count()['index'] < 4)])
drop_indices = [not aapl['Date'][i] in drop_dates for i in range(len(aapl))]

In [6]:
## now all dates have 4 <= i < 12 articles
aapl_processed = aapl[drop_indices]

In [7]:
sorted_dates  = sorted(aapl_processed['Date'].unique())
num_sequences = len(sorted_dates[4:])
num_sequences

792

In [58]:
#pad processed df to include kappa entries for each date
pd.options.mode.chained_assignment = None

padded_aapl_proc = aapl_processed.copy(deep = True)

for date in aapl_processed['Date'].unique(): 
    df_date = aapl_processed[aapl_processed['Date'] == date]
    
    if len(df_date) <  kappa: 
        row_append = df_date.head(1)
        #change value and key vec to be 0 for appended rows
        row_append.iloc[:, row_append.columns.get_loc('K0') : row_append.columns.get_loc('V299') + 1] = 0
        
        #temp variable to fix annpying pandas append tendencies
        temp = padded_aapl_proc
        for i in range(kappa - len(df_date)): 
            temp = temp.append(row_append, ignore_index = True)
            
        padded_aapl_proc = temp
    

In [72]:
padded_aapl_proc = padded_aapl_proc.sort_values('Date')

In [73]:
len(padded_aapl_proc)/12

796.0

Now that we have processed our data to include only robust inputs, let's do a quick refresher of what your initial input to the neural network is supposed to look like, and what dimensions it will have. Our key vectors for day $t$ are  $K_i^{(t)} \in \mathbb{R}^{64}$, and we have at most $\kappa$ articles per day. Thus, for any given day $t$, we can treat the input as $\begin{bmatrix} K_1^{(t)} & \cdots & K_\kappa^{(t)} \end{bmatrix} \in \mathbb{R}^{64 \times \kappa}$. Since our network uses five market vectors ($m^{(t - 4)}, \ldots, m^{(t)}$) for predicting stock price movement on any given day, we must pass in a sequence of $\kappa \cdot 5$ key vectors. 

So each input looks like $\begin{bmatrix} K_1^{(t - 4)} & \cdots & K_\kappa^{(t - 4)} & \cdots \cdots & K_1^{(t)} & \cdots & K_\kappa^{(t)} \end{bmatrix} \in \mathbb{R}^{64 \times 5\kappa}$. Then, assuming we have $k$ such datapoints (or in our case, $5$ day sequences) in our training dataset, our input is thus:
$\begin{bmatrix} 
K_1^{(t_1 - 4)} & \cdots & K_\kappa^{(t_1 - 4)} & \cdots \cdots & K_1^{(t_1)} & \cdots & K_\kappa^{(t_1)} \\
\vdots & & \vdots & & \vdots & & \vdots \\
K_1^{(t_k - 4)} & \cdots & K_\kappa^{(t_k - 4)} & \cdots \cdots & K_1^{(t_k)} & \cdots & K_\kappa^{(t_k)}
\end{bmatrix} \in \mathbb{R}^{64k \times 5\kappa}$
where each row represents a different sequence of $5$ days for the stock key vectors.

In [62]:
k = num_sequences
X_in = np.zeros((64*k, 5*kappa))

In [63]:
#iterate through rows, step size of 64 to account for size of key vectors
for i in range(0, 64*num_sequences, 64): 
    dates = sorted_dates[i : i + 5]
    
    #counter to keep track of column index
    counter = 0
    for date in dates: 
        df = aapl_processed[aapl_processed['Date'] == date]
        sub_mat = np.array(df.iloc[:, df.columns.get_loc('K0') : df.columns.get_loc('K63') + 1]).T
        X_in[i : i + 64, counter : counter + sub_mat.shape[1]] = sub_mat
        
        #increment by kappa to go to next day in sequence
        counter += kappa
        
    

In [64]:
64*k

50688

In [65]:
X_in.shape

(50688, 60)

In [66]:
X_in

array([[ 0.84720065,  1.99831985,  1.19001225, ...,  1.27729124,
         0.        ,  0.        ],
       [ 0.01052767, -0.82558529, -0.4053802 , ..., -1.03439441,
         0.        ,  0.        ],
       [-0.99316981, -1.1317625 , -1.35804945, ..., -1.81819254,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

### 1) b) Generating classifier labels

The classifier aims to predict price movement - in this notebook, we define movement in terms of log returns, the difference in log prices for a given day t, and a preceding day, t-1. Note that due to the nature of the dataset, we do not have a perfectly contiguous sequence of days, however, we use the next best approximation (i.e. t-2, if it is available, in place of t-1). Using these log returns, we binarize price movement: if returns are > 0 on day t, then $ y^t $ = 1, else 0. 

You can find historical stock price data on finance.yahoo.com, and use close prices to calculate log returns

In [80]:
prices = pd.read_csv('AAPL.csv')
prices['log_returns'] = np.log(prices['Close']) - np.log(prices['Close'].shift(1))

In [83]:
#change date format
prices['Date'] = prices['Date'].apply(lambda x: int(x.replace('-', '')))

In [84]:
prices.head()

,Date,Open,High,Low,Close,Adj Close,Volume,log_returns
0,20110131,11.992857,12.144286,11.939285,12.118571,10.369199,377246800,NaN
1,20110201,12.189285,12.344643,12.177857,12.322500,10.543692,426633200,0.016688
2,20110202,12.301785,12.330358,12.269643,12.297143,10.521996,258955200,-0.002060
3,20110203,12.278571,12.294286,12.091071,12.265715,10.495105,393797600,-0.002559
4,20110204,12.272857,12.382143,12.268214,12.375000,10.588613,321840400,0.008870


In [107]:
#generate labels for each of the k sequences for the AAPL stock
labels = []
for date in sorted_dates[4:]:
    log_ret = prices[prices['Date'] == date]['log_returns'].values
    
    if (len(log_ret) == 0): 
        labels.append(np.random.choice([1, 0]))
        continue
    
    if (log_ret[0]) > 0: 
        labels.append(1)
    else: 
        labels.append(0)
    

In [108]:
len(labels)

792

### 2) Building the pre-classifier model

Before building our main classifier, recall that we need to do some preprocessing to get from our input above to the market vectors that are being used in the classifier. The documentation for tensorflow and Keras will help a lot with some of the manipulations required for this section. In particular, the documentation for `Lambda` layers, `Dense` layers, `Concatenate` layers, `keras.activations.softmax` (which we have imported for you), and `tf.split` may be helpful. The outline of the necessary preprocessing steps is as follows:

1. As above, the input layer is of the form <br>
$\begin{bmatrix} 
K_1^{(t_1 - 4)} & \cdots & K_\kappa^{(t_1 - 4)} & \cdots \cdots & K_1^{(t_1)} & \cdots & K_\kappa^{(t_1)} \\
\vdots & & \vdots & & \vdots & & \vdots \\
K_1^{(t_k - 4)} & \cdots & K_\kappa^{(t_k - 4)} & \cdots \cdots & K_1^{(t_k)} & \cdots & K_\kappa^{(t_k)}
\end{bmatrix} \in \mathbb{R}^{64k \times 5\kappa}$
<br>

2. By treating the stock embedding $s$ as a weight from the input layer to the first hidden layer, generate layer 1 of the form <br>
$\begin{bmatrix} 
score_1^{(t_1 - 4)} & \cdots & score_\kappa^{(t_1 - 4)} & \cdots \cdots & score_1^{(t_1)} & \cdots & score_\kappa^{(t_1)} \\
\vdots & & \vdots & & \vdots & & \vdots \\
score_1^{(t_k - 4)} & \cdots & score_\kappa^{(t_k - 4)} & \cdots \cdots & score_1^{(t_k)} & \cdots & score_\kappa^{(t_k)}
\end{bmatrix} \in \mathbb{R}^{k \times 5\kappa}$ <br>
*Hint:* Remember, we define $score_i^{(t)} = K_i^{(t)} \cdot s$. Moreover, look at the documentation for applying Dense layers to rank $> 2$ tensors in Keras and see if you need to modify the input in some way before the input layer.
<br>

3. Apply softmax activations appropriately to generate layer 2 of the form <br>
$\begin{bmatrix} 
\alpha_1^{(t_1 - 4)} & \cdots & \alpha_\kappa^{(t_1 - 4)} & \cdots \cdots & \alpha_1^{(t_1)} & \cdots & \alpha_\kappa^{(t_1)} \\
\vdots & & \vdots & & \vdots & & \vdots \\
\alpha_1^{(t_k - 4)} & \cdots & \alpha_\kappa^{(t_k - 4)} & \cdots \cdots & \alpha_1^{(t_k)} & \cdots & \alpha_\kappa^{(t_k)}
\end{bmatrix} \in \mathbb{R}^{k \times 5\kappa}$ <br>
*Hint:* Remember, we define $\alpha_i^{(t)} = \displaystyle \frac{\exp(score_i^{(t)})}{\sum_{j \in [\kappa], j \neq i}exp(score_j^{(t)})}$.
<br>

4. Finally, the last layer before the classifier, layer 3, contains the market vectors and must be of the form <br>
$\begin{bmatrix} 
m^{(t_1 - 4)} & \cdots & m^{(t_1)}\\ 
\vdots & & \vdots \\ 
m^{(t_k)} & \cdots & m^{(t_k)}
\end{bmatrix} \in \mathbb{R}^{300k \times 5}$ <br>
*Hint:* Remember, we define $m_i^{(t)} = \displaystyle \sum_{i \in [\kappa]} \alpha_i^{(t)} V_i^{(t)}$. You may find the custom activation function that we have defined for you helpful.

In [110]:
[1, 2] + [3, 4]

[1, 2, 3, 4]

In [111]:
from keras import backend as K

def market_activation(v):
    '''
    Input: Matrix v of shape (300k, kappa) whose columns represent value vectors
    Returns: Function custom_sum(x)
    '''
    def custom_sum(x):
        '''
        Input: Tensor x of shape (None, k, kappa) with alpha values from layer 3
        Returns: Tensor m of shape (None, 300k, 1) that represents the market vectors as defined above
        '''
        k = x.shape[1]
        kappa = x.shape[2]
        cols = []
        for i in range(k):
            cols.append(sum([x[:, i, j] * v[:, j] for j in range(kappa)]))
        m = np.vstack(cols)
        return m
    
    return custom_sum

In [68]:
from keras.layers import Activation, Input, Dense, GRU, Bidirectional, Lambda
from keras.models import Model
from keras.layers.merge import Concatenate
from keras.activations import softmax

In [69]:
#transform input by transposing each key vector in the above matrix, i.e. 64k x 5kappa -> k x (64 * 5kappa)
rows = []
for i in range(0, 64 * k, 64):
    cols = [X_in[i : i + 64, j] for j in range(5 * kappa)]
    to_row = [c.T for c in cols]
    rows.append(np.array(to_row).reshape(1, 64 * 5 * kappa))
X_in_mod = np.vstack(rows)

x = Input(shape = X_in_mod.shape, name="Input")

In [70]:
cols = tf.split(x, num_or_size_splits = 5 * kappa, axis=2)
#each tensor in cols has shape (None, k, 64)

#create (5 * kappa) dense layers, 1 for each tensor in cols, and concatenate them together to get layer 1
dense_layers = []
for t in cols:
    lambd_layer = Lambda(lambda x:x)(t)
    dense_layers.append(Dense(1, use_bias=False)(lambd_layer))

layer1 = Concatenate(name="Layer1")(dense_layers)

In [71]:
cols = tf.split(layer1, num_or_size_splits = 5, axis=2)
#each tensor in cols has shape (None, k, kappa)

#create 5 softmax activations, 1 for each tensor in cols, and concatenate them together to get layer 2
softmax_layers = []
for t in cols:
    lambd_layer = Lambda(lambda x:x)(t)
    softmax_layers.append(softmax(lambd_layer, axis=2))

layer2 = Concatenate(name="Layer2")(softmax_layers)

In [ ]:
cols = tf.split(layer1, num_or_size_splits = 5, axis=2)
#each tensor in cols has shape (None, k, kappa)

#create 5 market activations, 1 for each tensor in cols, and concatenate them together to get layer 3

#counter to keep track of start index for dates
counter = 0
market_layers = []
for t in cols:
    lambd_layer = Lambda(lambda x:x)(t)
    
    dates = sorted_dates[counter:792 + counter]
    
    v_inp = np.array([])
    
    for date in dates: 
        df = padded_aapl_proc[padded_aapl_proc['Date'] == date]
        value_vecs = np.array(padded_aapl_proc.iloc[:, df.columns.get_loc('V0') : df.columns.get_loc('V299') + 1]).T
        
        if len(v_inp) == 0: 
            v_inp = value_vecs
    
        else: 
            v_inp = np.vstack((v_inp, value_vecs))
            
    counter += 1
    market_layers.append(market_activation(v_inp)(t))

layer3 = Concatenate(name="Layer3")(market_layers)

'''
for i in range(num_sequences*5): 
    df = aapl_processed.loc[i:i+kappa, :]
    value_vecs = np.array(df.iloc[:, df.columns.get_loc('V0') : df.columns.get_loc('V299') + 1]).T
    value_vecs = value_vecs.reshape(value_vecs.shape[0] * value_vecs.shape[1], 1)
    print(value_vecs.shape)
    
    alphas_arr = alphas[i]
    
    inp = tf.concat(alphas_arr, value_vecs) 
    print(inp.shape)
    #need to check how to pass multiple inputs into Dense layer
    linear_sums.append(Activation(custom_sum)(inp))
'''

In [46]:
model = Model(inputs = x, outputs = layer2)
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 792, 3840)]  0                                            
__________________________________________________________________________________________________
tf_op_layer_split_4 (TensorFlow [(None, 792, 64), (N 0           Input[0][0]                      
__________________________________________________________________________________________________
lambda_240 (Lambda)             (None, 792, 64)      0           tf_op_layer_split_4[0][0]        
__________________________________________________________________________________________________
lambda_241 (Lambda)             (None, 792, 64)      0           tf_op_layer_split_4[0][1]        
____________________________________________________________________________________________

In [ ]:
flattened_alphas = Concatenate([linear_sums[i] for i in range(len(linear_sums))])
bigru = Bidirectional(GRU(num_sequences, activation = 'relu'))(flattened_alphas)
pred = Dense(num_sequences, activation = 'sigmoid')(bigru)

model = Model(inputs = x, outputs = pred)